In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd
import numpy as np
import re
import nibabel as nib
from nilearn.plotting import plot_stat_map
import pickle

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
os.chdir('/Users/zeynepenkavi/Documents/RangelLab/NovelVsRepeated/fmri/analysis/01_level1/cluster_scripts')

In [3]:
subnum = '601'
session = '02'
task = 'binaryChoice'
runnum = '03'
mnum = 'model1'

data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'
out_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri/derivatives/nilearn/glm/level1/%s/%s'%(task, mnum)

In [4]:
from level1_utils import make_level1_design_matrix

In [ ]:
make_level1_design_matrix(subnum, session, task, runnum, mnum, data_path, hrf_model = 'spm', drift_model='cosine')

In [ ]:
subnum = '601'
session = '01'
task = 'yesNo'
runnum = '02'
mnum = 'model1'

make_level1_design_matrix(subnum, session, task, runnum, mnum, data_path, hrf_model = 'spm', drift_model='cosine')

In [5]:
from compute_contrasts_utils import make_basic_contrasts

In [6]:
design_matrix = make_level1_design_matrix(subnum, session, task, runnum, mnum, data_path, hrf_model = 'spm', drift_model='cosine')

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


In [7]:
contrasts = make_basic_contrasts(design_matrix)
contrasts

{'choiceCorrectHT_st': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 'choiceCorrectRE_st': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 'choiceIncorrectHT_st': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 'choiceLeft_st': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
contrasts['valHT-valRE'] = contrasts['valHT_par'] - contrasts['valRE_par']
contrasts['valHT-valRE'] = contrasts['valRE_par'] - contrasts['valHT_par']
contrasts['rewardHT-rewardRE'] = contrasts['rewardHT_par'] - contrasts['rewardRE_par']
contrasts['rewardRE-rewardHT'] = contrasts['rewardRE_par'] - contrasts['rewardHT_par']
contrasts